# Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import warnings
import re

from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)

# Importação dos arquivos

In [832]:
pd.set_option('display.max_columns', None)

In [833]:
df_applicants = pd.read_json('applicants/applicants.json', orient = 'index')

infos_basicas = pd.json_normalize(df_applicants['infos_basicas'])
informacoes_pessoais = pd.json_normalize(df_applicants['informacoes_pessoais'])
informacoes_profissionais = pd.json_normalize(df_applicants['informacoes_profissionais'])
formacao_e_idiomas = pd.json_normalize(df_applicants['formacao_e_idiomas'])
cargo_atual = pd.json_normalize(df_applicants['cargo_atual'])

df_applicants = pd.concat([infos_basicas, informacoes_pessoais, informacoes_profissionais, formacao_e_idiomas, cargo_atual, df_applicants['cv_pt']], axis=1)

df_applicants.rename(columns={'codigo_profissional': 'id_candidato'}, inplace=True)
df_applicants.dropna(subset='id_candidato', inplace=True)

df_applicants.head(5)

,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,id_candidato,nome,data_aceite,nome,cpf,fonte_indicacao,email,email_secundario,data_nascimento,telefone_celular,telefone_recado,sexo,estado_civil,pcd,endereco,skype,url_linkedin,facebook,download_cv,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,qualificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,instituicao_ensino_superior,cursos,ano_conclusao,outro_curso,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato,cv_pt
0,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,31000,Carolina Aparecida,Cadastro anterior ao registro de aceite,Carolina Aparecida,,:,carolina_aparecida@gmail.com,,0000-00-00,(11) 97048-2708,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,31001,Eduardo Rios,Cadastro anterior ao registro de aceite,Eduardo Rios,,Outros: Contato do RH,eduardo_rios@hotmail.com,,28-12-1994,(11) 93723-4396,,Feminino,Solteiro,Não,são paulo,,,,NaN,Analista Administrativo,Administrativa,,,,1900,,NaN,NaN,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"\nexperiência em análise, desenvolvimento e im..."
2,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,31002,Pedro Henrique Carvalho,Cadastro anterior ao registro de aceite,Pedro Henrique Carvalho,,Anúncio:,pedro_henrique_carvalho@gmail.com,,12-12-1988,(11) 92399-9824,,Feminino,Solteiro,Não,são paulo,,,,NaN,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,"2.500,00",,NaN,NaN,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,,Administração de Empresas,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n
3,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,31003,Thiago Barbosa,Cadastro anterior ao registro de aceite,Thiago Barbosa,,Site de Empregos: Infojobs,thiago_barbosa@hotmail.com,,08-05-1992,(11) 98100-1727,,Feminino,Casado,Não,são paulo,,,,NaN,Área administrativa,Administrativa,,,,"1100,00",,NaN,NaN,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n
4,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,31004,Diogo das Neves,Cadastro anterior ao registro de aceite,Diogo das Neves,,:,diogo_das_neves@hotmail.com,,31-12-1969,(11) 92517-2678,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [835]:
df_jobs = pd.read_json('jobs/jobs.json', orient='index')

df_info_basicas = pd.json_normalize(df_jobs['informacoes_basicas'])
df_perfil_vaga = pd.json_normalize(df_jobs['perfil_vaga'])
df_beneficios = pd.json_normalize(df_jobs['beneficios'])

df_jobs = pd.concat([pd.DataFrame(df_jobs.index), df_info_basicas, df_perfil_vaga, df_beneficios], axis=1)

df_jobs.rename(columns={0:'id_vaga'}, inplace=True)

df_jobs.head(5)

,id_vaga,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone,data_inicial,data_final,nome_substituto,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,18-05-2021,17-01-2022,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,


In [836]:
df_prospects = pd.read_json('prospects/prospects.json', orient='index')

df_prospects = df_prospects.explode('prospects')

prospects = pd.json_normalize(df_prospects['prospects'])
df_prospects = pd.concat([pd.DataFrame(df_prospects.index), prospects], axis=1)

df_prospects.rename(columns={0:'codigo_vaga', 'codigo': 'codigo_candidato', 'situacao_candidado': 'situacao_candidato'}, inplace=True)

df_prospects.head(10)

,codigo_vaga,nome,codigo_candidato,situacao_candidato,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4532,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4533,Arthur Almeida,26338,Contratado pela Decision,29-04-2021,18-05-2021,,Stella Vieira
6,4533,Dante Sampaio,24645,Desistiu,27-04-2021,27-04-2021,Profissional desistiu da vaga. Motivo : Na ver...,Yasmin da Rosa
7,4534,Ana Luiza Vieira,26361,Documentação PJ,28-04-2021,11-05-2021,Aguardando confirmação de inicio _,Manuella Carvalho
8,4534,Isabella da Cruz,26205,Desistiu,23-04-2021,30-04-2021,"profissional não atende ou responde ligações, ...",Ana Camargo
9,4534,Maria Helena Peixoto,26003,Não Aprovado pelo Cliente,08-04-2021,16-04-2021,"""Conversando com a candidata, foi exposto que ...",Carolina Araújo


# Seleção e Engenharia de Features

## Applicants

In [837]:
df_applicants.sample(5)

,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,id_candidato,nome,data_aceite,nome,cpf,fonte_indicacao,email,email_secundario,data_nascimento,telefone_celular,telefone_recado,sexo,estado_civil,pcd,endereco,skype,url_linkedin,facebook,download_cv,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,qualificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,instituicao_ensino_superior,cursos,ano_conclusao,outro_curso,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato,cv_pt
23950,,(17) 93129-1631,,31-03-2021 17:54:05,Ana Camargo,dr._theo_cavalcante@hotmail.com,,,31-03-2021 17:54:05,25859,Dr. Theo Cavalcante,Cadastro anterior ao registro de aceite,Dr. Theo Cavalcante,,:,dr._theo_cavalcante@hotmail.com,,0000-00-00,(17) 93129-1631,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,a quem possa interessar.\nprezados senhores:\n...
18524,,(31) 92414-6977,"Analista, supervisão, assistente, auxiliar",23-02-2021 22:07:13,Dra. Luara Siqueira,sra._catarina_casa_grande@gmail.com,"Contagem, Minas Gerais",Indeed,24-02-2021 21:48:33,24717,Sra. Catarina Casa Grande,Cadastro anterior ao registro de aceite,Sra. Catarina Casa Grande,,Indeed:,sra._catarina_casa_grande@gmail.com,,12-11-1973,(31) 92414-6977,,Masculino,Casado,Sim,minas gerais,,,,NaN,"Analista, supervisão, assistente, auxiliar","Administrativa, Financeira/Controladoria, Gest...",,,,6500,,NaN,NaN,Pós Graduação Completo,Básico,Nenhum,-,,Finanças/Controladoria,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\natualmente estou participando de implementaç...
21862,,(11) 94350-7726,,17-06-2021 16:51:55,Carlos Eduardo Ribeiro,dra._hellena_nogueira@hotmail.com,,,17-06-2021 16:51:55,27524,Dra. Hellena Nogueira,Cadastro anterior ao registro de aceite,Dra. Hellena Nogueira,,:,dra._hellena_nogueira@hotmail.com,,0000-00-00,(11) 94350-7726,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"brasileiro, 40 anos\ndados pessoais\nnão tenho..."
38642,,(55) 98591-8996,"VENDAS, MARKETING E PROCESSOS",26-05-2020 18:47:50,pelo próprio candidato,sr._caio_siqueira@gmail.com,"Osasco, São Paulo",Nosso site,26-05-2020 18:47:50,19741,Sr. Caio Siqueira,Cadastro anterior ao registro de aceite,Sr. Caio Siqueira,,Nosso site:,sr._caio_siqueira@gmail.com,,12-05-1958,(55) 98591-8996,,Masculino,Casado,Não,são paulo,,,,NaN,"VENDAS, MARKETING E PROCESSOS",Novos Negócios e Parcerias,,,,7500,,NaN,NaN,Pós Graduação Completo,Básico,Básico,-,,Marketing/Vendas,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\n\n\ncontato\n\nprincipais competências re...
34635,,(19) 98320-1783,,30-07-2021 12:45:13,pelo próprio candidato,arthur_gabriel_duarte@hotmail.com,"Campinas, São Paulo",Outros,30-07-2021 12:45:13,28668,Arthur Gabriel Duarte,Cadastro anterior ao registro de aceite,Arthur Gabriel Duarte,,Outros: Linkedin,arthur_gabriel_duarte@hotmail.com,,18-02-1988,(19) 98320-1783,,Masculino,Solteiro,Não,são paulo,,,,NaN,,TI - Desenvolvimento/Programação,,,,"6500,00",,NaN,NaN,Pós Graduação Completo,Intermediário,Básico,-,,Informática,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"25 anos , solteira\n\n\n\n\ndados pessoais\npa..."


### Seleção das features

In [838]:
df_applicants_features = df_applicants[['id_candidato', 'titulo_profissional', 'area_atuacao', 'nivel_academico','nivel_ingles', 'conhecimentos_tecnicos', 'cv_pt']]
df_applicants_features.head()

,id_candidato,titulo_profissional,area_atuacao,nivel_academico,nivel_ingles,conhecimentos_tecnicos,cv_pt
0,31000,,,,,,
1,31001,Analista Administrativo,Administrativa,Ensino Superior Incompleto,Nenhum,,"\nexperiência em análise, desenvolvimento e im..."
2,31002,Administrativo | Financeiro,Administrativa,Ensino Superior Completo,Intermediário,,\n
3,31003,Área administrativa,Administrativa,Ensino Superior Incompleto,Nenhum,,\n
4,31004,,,,,,


In [839]:
df_applicants_features['conhecimentos_tecnicos'].replace('', np.nan, inplace=True)
df_applicants_features['cv_pt'].replace('', np.nan, inplace=True)

In [840]:
df_applicants_features['tem_cv'] = df_applicants_features['cv_pt'].notna().astype(int)
df_applicants_features['tem_conhecimentos_tecnicos'] = df_applicants_features['conhecimentos_tecnicos'].notna().astype(int)

### Categorização da Área de Atuação [area_atuacao]

In [ ]:
# Categorização das áreas de atuação classifcadas por IA
classes_area_atuacao = pd.read_csv('area_atuacao_classificada.csv')
classes_area_atuacao = classes_area_atuacao.drop_duplicates(subset=['Area de Atuacao'])

df_applicants_features = df_applicants_features.merge(classes_area_atuacao, left_on='area_atuacao', right_on='Area de Atuacao', how='left')

df_applicants_features.drop(['area_atuacao', 'Area de Atuacao', '0'], axis=1, inplace=True)
df_applicants_features.rename(columns={'Generalized Area': 'area_atuacao'}, inplace=True)

### Tratamento do Título Profissional [titulo_profissional]

In [844]:
def classificar_titulo(titulo):
    """
    Classifica um título profissional com base em palavras-chave,
    verificando do nível mais alto para o mais baixo.
    """
    # 1. Tratar valores nulos primeiro
    if pd.isna(titulo):
        return 'Sem informação'

    # 2. Padronizar o texto para minúsculas
    titulo_lower = str(titulo).lower()

    # 3. Aplicar as regras na ordem de senioridade (do mais alto para o mais baixo)
    if 'gerente' in titulo_lower:
        return 'Gerente'
    if 'coordenador' in titulo_lower or 'coord.' in titulo_lower:
        return 'Coordenador'
    if 'supervisor' in titulo_lower:
        return 'Supervisor'
    if 'líder' in titulo_lower or 'lider' in titulo_lower or 'lead' in titulo_lower:
        return 'Líder'
    if 'especialista' in titulo_lower or 'specialist' in titulo_lower:
        return 'Especialista'
    if 'sênior' in titulo_lower or 'senior' in titulo_lower or ' sr' in titulo_lower:
        return 'Sênior'
    if 'pleno' in titulo_lower or ' pl' in titulo_lower:
        return 'Pleno'
    # 'Analista' vem depois dos níveis para não capturar "Analista Sênior" etc.
    if 'analista' in titulo_lower or 'analyst' in titulo_lower:
        return 'Analista'
    if 'júnior' in titulo_lower or 'junior' in titulo_lower or ' jr' in titulo_lower:
        return 'Júnior'
    if 'trainee' in titulo_lower:
        return 'Trainee'
    if 'técnico' in titulo_lower or 'tecnico' in titulo_lower:
        return 'Técnico de Nível Médio'
    if 'assistente' in titulo_lower:
        return 'Assistente'
    if 'auxiliar' in titulo_lower:
        return 'Auxiliar'
    if 'aprendiz' in titulo_lower:
        return 'Aprendiz'

    # 4. Se nenhuma regra corresponder, retorna None (não classificado)
    return 'Sem informação'

In [845]:
df_applicants_features['titulo_profissional'] = df_applicants_features['titulo_profissional'].apply(classificar_titulo)
df_applicants_features.rename(columns={'titulo_profissional': 'nivel_profissional'}, inplace=True)

### Ordinal Label Enconding

In [846]:
# Mapeamento do nível profissional dos candidatos
mapeamento_nivel_profissional_applicants = {
    'Sem Informação': -1,
    'Aprendiz': 0,
    'Auxiliar': 1,
    'Assistente': 1,
    'Técnico de Nível Médio': 2,
    'Júnior': 3,
    'Trainee': 4,
    'Pleno': 4,
    'Analista': 4,
    'Sênior': 5,
    'Especialista': 6,
    'Líder': 7,
    'Supervisor': 7,
    'Coordenador': 8,
    'Gerente': 9
}

In [847]:
df_applicants_features['nivel_profissional'] = df_applicants_features['nivel_profissional'].map(mapeamento_nivel_profissional_applicants)

In [848]:
df_applicants_features['nivel_academico'].replace('','Sem informação', inplace=True)
df_applicants_features['nivel_academico'] = df_applicants_features['nivel_academico'].str.replace('Cursando', 'Incompleto', regex=False)

In [849]:
# Lista ordenada do menor para o maior nível acadêmico
ordem_nivel_academico_applicants = [
    'Sem informação',
    'Ensino Fundamental Incompleto',
    'Ensino Fundamental Completo',
    'Ensino Médio Incompleto',
    'Ensino Médio Completo',
    'Ensino Técnico Incompleto',
    'Ensino Técnico Completo',
    'Ensino Superior Incompleto',
    'Ensino Superior Completo',
    'Pós Graduação Incompleto',
    'Pós Graduação Completo',
    'Mestrado Incompleto',
    'Mestrado Completo',
    'Doutorado Incompleto',
    'Doutorado Completo'
]

In [850]:
mapeamento_nivel_academico_applicants = {nivel: i-1 for i, nivel in enumerate(ordem_nivel_academico_applicants)}

In [851]:
df_applicants_features['nivel_academico'] = df_applicants_features['nivel_academico'].map(mapeamento_nivel_academico_applicants)

In [852]:
df_applicants_features['nivel_ingles'].replace('', "Nenhum", inplace=True)

In [853]:
# Lista ordenada do menor para o maior nível de inglês
ordem_nivel_ingles = [
    'Nenhum',
    'Básico',
    'Intermediário',
    'Técnico',
    'Avançado',
    'Fluente',
]

In [854]:
mapeamento_nivel_ingles = {nivel: i for i, nivel in enumerate(ordem_nivel_ingles)}

In [855]:
df_applicants_features['nivel_ingles'] = df_applicants_features['nivel_ingles'].map(mapeamento_nivel_ingles)

### Embeddings das informações não estruturadas [conhecimentos_tecnicos, cv_pt]

In [856]:
def limpar_texto(texto):
    """
    Esta função recebe um texto com HTML e 'sujeiras' e retorna um texto limpo.
    """
    if not isinstance(texto, str):
        return "" # Retorna string vazia se o dado não for texto (ex: NaN)

    # Remove todas as tags HTML, substituindo-as por um espaço
    texto_limpo = re.sub(r'<[^>]+>', ' ', texto)
    
    # Remove caracteres de nova linha, substituindo por um espaço
    texto_limpo = texto_limpo.replace('\n', ' ')
    
    # Remove múltiplos espaços em sequência, substituindo por um único espaço
    texto_limpo = re.sub(r'\s+', ' ', texto_limpo)
    
    # Remove espaços no início e no fim do texto
    texto_limpo = texto_limpo.strip()
    texto_limpo = texto_limpo.lower()
    
    return texto_limpo

In [857]:
# Tratamento e limpeza dos textos
df_applicants_features['cv_pt'] = df_applicants_features['cv_pt'].apply(limpar_texto)
df_applicants_features['conhecimentos_tecnicos'] = df_applicants_features['conhecimentos_tecnicos'].apply(limpar_texto)

In [858]:
# Inicialização do modelo
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

embedding_dim = model.get_sentence_embedding_dimension()

In [859]:
cv_pt = df_applicants_features['cv_pt'].to_list()

conhecimentos_tecnicos = df_applicants_features['conhecimentos_tecnicos'].to_list()

In [860]:
embedding_cv_pt = model.encode(cv_pt, show_progress_bar=True)

Batches: 100%|██████████| 1328/1328 [15:25<00:00,  1.44it/s]


In [861]:
embedding_conhecimentos_tecnicos = model.encode(conhecimentos_tecnicos, show_progress_bar=True)

Batches: 100%|██████████| 1328/1328 [02:02<00:00, 10.80it/s]


In [862]:
embedding_cv_pt[df_applicants_features['tem_cv'] == 0] = np.zeros(embedding_dim)
embedding_conhecimentos_tecnicos[df_applicants_features['tem_conhecimentos_tecnicos'] == 0] = np.zeros(embedding_dim)

In [863]:
df_applicants_features['embedding_cv_pt'] = list(embedding_cv_pt)
df_applicants_features['embedding_conhecimentos_tecnicos'] = list(embedding_conhecimentos_tecnicos)

## Jobs

In [864]:
df_jobs.head()

,id_vaga,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone,data_inicial,data_final,nome_substituto,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,18-05-2021,17-01-2022,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,


In [865]:
df_jobs.rename(columns={'nivel profissional': 'nivel_profissional', 'areas_atuacao': 'area_atuacao'}, inplace=True)

### Seleção das Features

In [866]:
df_jobs_features = df_jobs[['id_vaga', 'nivel_profissional', 'nivel_academico', 'nivel_ingles', 'area_atuacao', 'principais_atividades', 'competencia_tecnicas_e_comportamentais']]
df_jobs_features.head()

,id_vaga,nivel_profissional,nivel_academico,nivel_ingles,area_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais
0,5185,Sênior,Ensino Superior Completo,Avançado,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...
1,5184,Sênior,Ensino Superior Completo,Fluente,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...
2,5183,Analista,Ensino Superior Completo,Nenhum,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...
3,5182,Analista,Ensino Superior Completo,Básico,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...
4,5181,Sênior,Ensino Superior Completo,Intermediário,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...


In [867]:
df_jobs_features['principais_atividades'].replace('', np.nan, inplace=True)
df_jobs_features['competencia_tecnicas_e_comportamentais'].replace('', np.nan, inplace=True)

In [868]:
df_jobs_features.dropna(subset=['principais_atividades', 'competencia_tecnicas_e_comportamentais'], inplace=True)

### Ordinal Label Encoding

In [869]:
mapeamento_nivel_profissional_jobs = {
    'Aprendiz': 0,
    'Auxiliar': 1,
    'Assistente': 1,
    'Técnico de Nível Médio': 2,
    'Júnior': 3,
    'Trainee': 4,
    'Pleno': 4,
    'Analista': 4,
    'Sênior': 5,
    'Especialista': 6,
    'Líder': 7,
    'Supervisor': 7,
    'Coordenador': 8,
    'Gerente': 9
}

In [870]:
df_jobs_features['nivel_profissional'] = df_jobs_features['nivel_profissional'].map(mapeamento_nivel_profissional_jobs)

In [871]:
# Lista ordenada do menor para o maior nível acadêmico
ordem_nivel_academico_jobs = [
    'Ensino Fundamental Completo',
    'Ensino Médio Incompleto',
    'Ensino Médio Completo',
    'Ensino Técnico Incompleto',
    'Ensino Técnico Cursando',
    'Ensino Técnico Completo',
    'Ensino Superior Incompleto',
    'Ensino Superior Cursando',
    'Ensino Superior Completo',
    'Pós Graduação Incompleto',
    'Pós Graduação Cursando',
    'Pós Graduação Completo',
    'Mestrado Cursando',
    'Mestrado Completo',
    'Doutorado Cursando',
    'Doutorado Completo'
]

In [872]:
mapeamento_nivel_academico_jobs = {nivel: i for i, nivel in enumerate(ordem_nivel_academico_jobs)}

In [873]:
df_jobs_features['nivel_academico'] = df_jobs_features['nivel_academico'].map(mapeamento_nivel_academico_jobs)

In [874]:
# Lista ordenada do menor para o maior nível de inglês
ordem_nivel_ingles = [
    'Nenhum',
    'Básico',
    'Intermediário',
    'Técnico',
    'Avançado',
    'Fluente',
]

In [875]:
mapeamento_nivel_ingles = {nivel: i for i, nivel in enumerate(ordem_nivel_ingles)}

In [876]:
df_jobs_features['nivel_ingles'] = df_jobs_features['nivel_ingles'].map(mapeamento_nivel_ingles)

### Categorização da Área de Atuação [areas_atuacao]

In [877]:
# Classificação da área de atuação em categorias
classes_area_atuacao_jobs = pd.read_csv('area_atuacao_jobs_classificada.csv')
classes_area_atuacao_jobs = classes_area_atuacao_jobs.drop_duplicates(subset=['areas_atuacao'])

df_jobs_features = df_jobs_features.merge(classes_area_atuacao_jobs, left_on='area_atuacao', right_on='areas_atuacao', how='left')

df_jobs_features.drop(columns=['id_vaga_y', 'area_atuacao'], axis=1, inplace=True)
df_jobs_features.rename(columns={'id_vaga_x': 'id_vaga', 'grupo': 'area_atuacao'}, inplace=True)

### Embeddings das informações não estruturadas [principais_atividades, competencia_tecnicas_e_comportamentais]

In [878]:
df_jobs_features['principais_atividades'] = df_jobs_features['principais_atividades'].apply(limpar_texto)
df_jobs_features['competencia_tecnicas_e_comportamentais'] = df_jobs_features['competencia_tecnicas_e_comportamentais'].apply(limpar_texto)

In [879]:
principais_atividades = df_jobs_features['principais_atividades'].to_list()
competencias = df_jobs_features['competencia_tecnicas_e_comportamentais'].to_list()

In [880]:
embedding_principais_atividades = model.encode(principais_atividades, show_progress_bar=True)

Batches: 100%|██████████| 440/440 [04:49<00:00,  1.52it/s]


In [881]:
embedding_competencias = model.encode(competencias, show_progress_bar=True)

Batches: 100%|██████████| 440/440 [04:41<00:00,  1.56it/s]


In [882]:
df_jobs_features['embedding_principais_atividades'] = list(embedding_principais_atividades)
df_jobs_features['embedding_competencias'] = list(embedding_competencias)

## Criação do DataFrame candidato-vaga


In [883]:
df_prospects.head()

,codigo_vaga,nome,codigo_candidato,situacao_candidato,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4532,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [884]:
df_prospects_temp = df_prospects[['codigo_vaga', 'codigo_candidato']]

df_prospects_temp.dropna(inplace=True)

In [885]:
df_prospects_temp = df_prospects_temp.merge(df_applicants_features, left_on='codigo_candidato', right_on='id_candidato', how='inner')

In [886]:
df_prospects_temp = df_prospects_temp.merge(df_jobs_features, left_on='codigo_vaga', right_on='id_vaga', how='inner', suffixes=(None, '_jobs'))

In [887]:
df_prospects_temp.drop(columns='cv_pt', axis=1, inplace=True)

In [888]:
df_prospects_temp['sim_cv_principais_atividades'] = df_prospects_temp.apply(
    lambda row: util.cos_sim(row['embedding_cv_pt'], row['embedding_principais_atividades']).item(), axis=1
)

In [889]:
df_prospects_temp['sim_conhecimentos_competencias'] = df_prospects_temp.apply(
    lambda row: util.cos_sim(row['embedding_conhecimentos_tecnicos'], row['embedding_competencias']).item(), axis=1
)

In [890]:
df_prospects_temp.head()

,codigo_vaga,codigo_candidato,id_candidato,nivel_profissional,nivel_academico,nivel_ingles,conhecimentos_tecnicos,tem_cv,tem_conhecimentos_tecnicos,area_atuacao,embedding_cv_pt,embedding_conhecimentos_tecnicos,id_vaga,nivel_profissional_jobs,nivel_academico_jobs,nivel_ingles_jobs,principais_atividades,competencia_tecnicas_e_comportamentais,areas_atuacao,area_atuacao_jobs,embedding_principais_atividades,embedding_competencias,sim_cv_principais_atividades,sim_conhecimentos_competencias
0,4530,25632,25632,NaN,-1,0,,1,0,NaN,"[-0.06571322, 0.04668367, -0.051871277, -0.042...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4530,4,8,0,- experiência comprovada em projetos de control-m,- experiência comprovada em projetos de control-m,TI - Desenvolvimento/Programação-,TI - Desenvolvimento,"[-0.1101493, 0.1515397, -0.2373129, 0.03142173...","[-0.1101493, 0.1515397, -0.2373129, 0.03142173...",0.410359,0.000000
1,4530,25529,25529,4.0,7,2,,1,0,TI - Multidisciplinar,"[0.04894809, 0.105838075, -0.10081045, -0.0313...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4530,4,8,0,- experiência comprovada em projetos de control-m,- experiência comprovada em projetos de control-m,TI - Desenvolvimento/Programação-,TI - Desenvolvimento,"[-0.1101493, 0.1515397, -0.2373129, 0.03142173...","[-0.1101493, 0.1515397, -0.2373129, 0.03142173...",0.493705,0.000000
2,4531,25364,25364,7.0,7,4,"- peoplesoft (peopletools 8.49, 8.53, 8.55, 8....",1,1,TI - Gestão/Projetos,"[0.12917836, 0.0866068, -0.082769156, -0.03844...","[-0.35935482, -0.17729728, 0.0034469012, 0.052...",4531,5,2,0,key skills required for the job are: peoplesof...,o recurso peoplesoft tem como responsabilidade...,Gestão e Alocação de Recursos de TI-,TI - Multidisciplinar,"[-0.106042996, -0.01572514, -0.044918414, -0.0...","[-0.4254325, 0.1501221, -0.031628404, 0.017136...",0.527758,0.695522
3,4531,25360,25360,NaN,-1,0,,1,0,NaN,"[-0.08601268, 0.03938286, -0.007746513, -0.083...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4531,5,2,0,key skills required for the job are: peoplesof...,o recurso peoplesoft tem como responsabilidade...,Gestão e Alocação de Recursos de TI-,TI - Multidisciplinar,"[-0.106042996, -0.01572514, -0.044918414, -0.0...","[-0.4254325, 0.1501221, -0.031628404, 0.017136...",0.526717,0.000000
4,4533,26338,26338,NaN,-1,0,,0,0,NaN,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",4533,5,2,3,arquiteto foco na área e automação. requerido ...,arquiteto foco na área e automação. requerido ...,Gestão e Alocação de Recursos de TI-,TI - Multidisciplinar,"[-0.12666267, 0.16972429, 0.08676297, -0.03039...","[-0.12666267, 0.16972429, 0.08676297, -0.03039...",0.000000,0.000000


In [891]:
df_prospects_features = df_prospects_temp[['codigo_vaga', 'codigo_candidato', 'nivel_profissional', 'nivel_profissional_jobs',
       'nivel_academico', 'nivel_academico_jobs', 'nivel_ingles', 'nivel_ingles_jobs', 'area_atuacao', 'area_atuacao_jobs',
       'sim_cv_principais_atividades', 'sim_conhecimentos_competencias']]

In [892]:
# Match entre o nível profissional exigido pela vaga e o nivel profissional do candidato
df_prospects_features['match_profissional'] = (abs(df_prospects_features['nivel_profissional'] - df_prospects_features['nivel_profissional_jobs']) <= 1).astype(int)

# Match entre o nível acadêmico exigido pela vaga e o nível acadêmico do candidato
df_prospects_features['match_academico'] = (df_prospects_features['nivel_academico'] >= df_prospects_features['nivel_academico_jobs']).astype(int)

# Match entre o nível de inglês exigido pela vaga e o nível de inglês do candidato
df_prospects_features['match_ingles'] = (df_prospects_features['nivel_ingles'] >= df_prospects_features['nivel_ingles_jobs']).astype(int)

# Match entre a área de atuação da vaga e do candidato
df_prospects_features['match_area'] = (df_prospects_features['area_atuacao'] == df_prospects_features['area_atuacao_jobs']).astype(int)

In [893]:
df_prospects_features['sim_cv_principais_atividades'] = df_prospects_features['sim_cv_principais_atividades'].clip(lower=0)
df_prospects_features['sim_conhecimentos_competencias'] = df_prospects_features['sim_conhecimentos_competencias'].clip(lower=0)

In [894]:
df_prospects_final = df_prospects_features[['codigo_vaga', 'codigo_candidato', 'match_profissional', 'match_academico', 'match_ingles', 'match_area',
                                            'sim_cv_principais_atividades', 'sim_conhecimentos_competencias']]

In [895]:
# Seleção das colunas para calcular o score final
score_cols = [
    'match_profissional',
    'match_academico',
    'match_ingles',
    'match_area',
    'sim_cv_principais_atividades',
    'sim_conhecimentos_competencias'
]

# Atribuição de pesos
pesos = [2, 1, 1, 2, 2, 2]

# DataFrame com as colunas de score
scores = df_prospects_final[score_cols]

# Cálculo do Match Score e inclusão no DataFrame Candidato-Vaga
df_prospects_final['match_score'] = np.average(scores, axis=1, weights=pesos)
df_prospects_final['compatibilidade'] = df_prospects_final['match_score'].apply(lambda x: f"{x:.2%}")

In [896]:
df_nomes = df_applicants[['id_candidato', 'nome']]
df_nomes.columns = ['codigo_candidato', 'nome_candidato', 'nome']
df_nomes.drop(['nome'], axis=1, inplace=True)

df_titulos_vagas = df_jobs[['id_vaga', 'titulo_vaga']].rename(columns={'id_vaga': 'codigo_vaga'})

In [899]:
df_prospects_final = pd.merge(df_prospects_final, df_nomes, on='codigo_candidato', how='left')

df_prospects_final = pd.merge(df_prospects_final, df_titulos_vagas, on='codigo_vaga', how='left')

In [900]:
ordem_colunas = ['codigo_vaga', 'titulo_vaga', 'codigo_candidato', 'nome_candidato', 
       'match_profissional', 'match_academico', 'match_ingles', 'match_area',
       'sim_cv_principais_atividades', 'sim_conhecimentos_competencias',
       'match_score', 'compatibilidade']

In [901]:
df_prospects_final = df_prospects_final[ordem_colunas]

In [902]:
df_prospects_final.sample(5)

,codigo_vaga,titulo_vaga,codigo_candidato,nome_candidato,match_profissional,match_academico,match_ingles,match_area,sim_cv_principais_atividades,sim_conhecimentos_competencias,match_score,compatibilidade
14802,933,Analista de Recrutamento TI,16020,Felipe Martins,0,0,0,0,0.654043,0.000000,0.130809,13.08%
43108,2078,Analista - AWS,1677,Milena Ramos,1,0,0,0,0.319265,0.470061,0.357865,35.79%
31289,8862,Assistente Administrativo / Controlador de Acesso,34934,Bianca Pires,0,1,1,0,0.000000,0.000000,0.200000,20.00%
24761,1253,Desenvolvedor Java - Angular Jr.,16346,Maya Lima,0,0,0,0,0.255608,0.000000,0.051122,5.11%
24837,2736,Analista de Automação de Testes (I),20477,Renan Pires,1,1,1,0,0.452887,0.488075,0.588193,58.82%


In [ ]:
# df_prospects_final.to_csv('df_prospects_compatibilidade.csv')